<a href="https://colab.research.google.com/github/tawsif67/ALBERT-with-FFNN/blob/main/VQA_draft_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install transformers
!pip install tqdm
!pip install openai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00


In [2]:
from transformers import ViltConfig
config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
print(config.label2id)

{'0': 82, '000': 3001, '1': 12, '1 4': 1990, '1 foot': 335, '1 hour': 1967, '1 in back': 3026, '1 in front': 2299, '1 in middle': 3059, '1 inch': 2146, '1 on left': 1723, '1 on right': 314, '1 way': 201, '1 world': 2360, '1 year': 2807, '1.00': 3000, '10': 103, '10 feet': 941, '10 inches': 2929, '10 years': 47, '100': 1161, '100 feet': 150, '100 year party ct': 3120, '1000': 1511, '101': 2789, '106': 2450, '10:00': 1661, '10:05': 1184, '10:08': 3117, '10:10': 519, '10:15': 2766, '10:20': 2177, '10:25': 2048, '10:30': 2410, '10:35': 3103, '10:40': 2040, '10:45': 3060, '10:50': 2880, '10:55': 2451, '11': 191, '11:00': 1907, '11:05': 2730, '11:10': 2646, '11:15': 2020, '11:20': 2141, '11:25': 3062, '11:30': 995, '11:35': 2198, '11:45': 2580, '11:50': 2790, '11:55': 186, '12': 396, '12 feet': 450, '120': 1973, '12:00': 1763, '12:05': 1759, '12:10': 2036, '12:15': 754, '12:20': 2754, '12:25': 2035, '12:28': 2033, '12:30': 1073, '12:35': 1651, '12:40': 3073, '12:45': 2245, '12:50': 2506, '12

In [3]:
!unzip -q "/content/images.zip"

# Create label2id

In [112]:
import csv

def create_dictionary_from_csv(csv_file_path):
    result_dict = {}
    label2id_dict= {}
    with open('/content/updated_output.csv', 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            label = int(row['label'])
            answer = row['answer']
            result_dict[label] = answer
            label2id_dict[answer] = label
    return result_dict

def label2id(csv_file_path):
    result_dict = {}
    label2id_dict= {}
    with open('/content/updated_output.csv', 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            label = int(row['label'])
            answer = row['answer']
            result_dict[label] = answer
            label2id_dict[answer] = label
    return label2id_dict

# Provide the path to your CSV file
csv_file_path = '/content'

# Call the function to create the dictionary
dictionary = create_dictionary_from_csv(csv_file_path)
label2id_dict = label2id(csv_file_path)
id2label_dict = dictionary
# Print the resulting dictionary
print(dictionary)

{0: 'Pink Block', 1: 'Purple Block', 2: 'Blue Block', 3: 'Gray Block', 4: 'Cyan Block', 5: 'Green Block'}


In [113]:
import pandas as pd

def create_unique_labels(csv_file):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file)

        # Delete the existing "label" column if it exists
        if 'label' in df.columns:
            df.drop(columns=['label'], inplace=True)

        # Create a mapping dictionary to assign unique numerical labels to each unique "answer" value
        unique_answers = df['answer'].unique()
        label_mapping = {answer: label for label, answer in enumerate(unique_answers)}

        # Create the new "label" column using the mapping
        df['label'] = df['answer'].map(label_mapping)

        # Save the updated DataFrame back to the CSV file
        df.to_csv("updated_output.csv", index=False)

        print("New 'label' column with unique numerical values has been created and saved as updated_output.csv.")
    except Exception as e:
        print(f"Error occurred: {e}")

# Usage example
if __name__ == "__main__":
    csv_file_path = "/content/sorted_output.csv"
    create_unique_labels(csv_file_path)

New 'label' column with unique numerical values has been created and saved as updated_output.csv.


# Data class

In [114]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [115]:
import torch
from PIL import Image
import pandas as pd
from transformers import BertTokenizer

df = pd.read_csv('/content/updated_output.csv')
questions = df['question']
#print(questions)
annotations = df['answer']
#print(annotations)
labels = df['label']
#print(labels)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class VQADataset(torch.utils.data.Dataset):
    """VQA (v2) dataset."""
    def __init__(self, df, processor, tokenizer):
    # def __init__(self, questions, annotations, processor):
        self.df = df
        self.questions = df['question']
        print(self.questions)
        self.annotations = df['answer']
        print(self.annotations)
        self.labels = df['label']
        print(self.labels)
        self.processor = processor
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # get image + text
        annotation = self.annotations[idx]
        questions = self.questions[idx]
        img_id = self.df['image'][idx]
        #print(img_id)
        image = Image.open(f"images/{img_id}.jpg")
        labels = self.labels[idx]
        #image.resize((128, 128))
        text = questions
        targets = torch.zeros(len(id2label_dict))
        targets[labels-1] = 1
        encoding = self.processor(image, text, padding="max_length", truncation=True, return_tensors="pt")
        # remove batch dimension
        for k,v in encoding.items():
          # print(k)
          encoding[k] = v.squeeze()
        #targets = torch.zeros(len(id2label_dict))
        tokens = tokenizer.tokenize(annotation)
        #tokens = [int(token) for token in tokens]
        #print(tokens)

        encoding["labels"] = targets
        #encoding["labels"] = torch.tensor(encoding["labels"])
        #encoding["labels"] = encoding["labels"].view(3129)
        #print(tokens)
        return encoding

In [83]:
targets = torch.zeros(len(config.id2label))
targets.shape

torch.Size([3129])

In [116]:
from transformers import ViltForQuestionAnswering

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",
                                                 num_labels=len(id2label_dict),
                                                 id2label=id2label_dict,
                                                 label2id=label2id_dict)
model.to(device)

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-mlm and are newly initialized: ['classifier.0.weight', 'classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.0.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=76

In [117]:
from transformers import ViltProcessor

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")

dataset = VQADataset(df=df,
                     processor=processor, tokenizer=tokenizer)

0                  Which object is closer to the middle?
1                  Which object is closer to the middle?
2                  Which object is closer to the middle?
3                  Which object is closer to the middle?
4               Which object is closest to the red bowl?
                             ...                        
419    Which block is farther from the middle, the pu...
420            Which object is closest to the blue bowl?
421        Which block is farther from the orange block?
422            Which object is closest to the blue bowl?
423              Which block is farther from the middle?
Name: question, Length: 424, dtype: object
0        Pink Block
1        Pink Block
2        Pink Block
3        Pink Block
4      Purple Block
           ...     
419    Purple Block
420      Pink Block
421      Pink Block
422      Pink Block
423    Purple Block
Name: answer, Length: 424, dtype: object
0      0
1      0
2      0
3      0
4      1
      ..
419    1
420   

In [118]:
processor.decode(dataset[0]['input_ids'])

'[CLS] which object is closer to the middle? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [119]:
labels = torch.nonzero(dataset[0]['labels']).squeeze().tolist()

## Define model

Here we define a `ViltForQuestionAnswering` model, with the weights of the body initialized from dandelin/vilt-b32-mlm, and a randomly initialized classification head. We also move it to the GPU, if it's available.

In [120]:
from torch.utils.data import DataLoader

def collate_fn(batch):
  input_ids = [item['input_ids'] for item in batch]
  pixel_values = [item['pixel_values'] for item in batch]
  attention_mask = [item['attention_mask'] for item in batch]
  token_type_ids = [item['token_type_ids'] for item in batch]
  labels = [item['labels'] for item in batch]

  # create padded pixel values and corresponding pixel mask
  encoding = processor.feature_extractor.pad_and_create_pixel_mask(pixel_values, return_tensors="pt")

  # create new batch
  batch = {}
  batch['input_ids'] = torch.stack(input_ids)
  batch['attention_mask'] = torch.stack(attention_mask)
  batch['token_type_ids'] = torch.stack(token_type_ids)
  batch['pixel_values'] = encoding['pixel_values']
  batch['pixel_mask'] = encoding['pixel_mask']
  # batch['pad_and_create_pixel_mask'] =
  batch['labels'] = torch.stack(labels)

  return batch

# train_dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=4, shuffle=True)
train_dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [121]:
batch = next(iter(train_dataloader))
print(vars(model))

{'training': False, '_parameters': OrderedDict(), '_buffers': OrderedDict(), '_non_persistent_buffers_set': set(), '_backward_pre_hooks': OrderedDict(), '_backward_hooks': OrderedDict(), '_is_full_backward_hook': None, '_forward_hooks': OrderedDict(), '_forward_hooks_with_kwargs': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_forward_pre_hooks_with_kwargs': OrderedDict(), '_state_dict_hooks': OrderedDict(), '_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_post_hooks': OrderedDict(), '_modules': OrderedDict([('vilt', ViltModel(
  (embeddings): ViltEmbeddings(
    (text_embeddings): TextEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(40, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (patch_embeddings): ViltPatchEmbeddings(
      (projection):

In [122]:
labels

5

In [98]:
for k,v in batch.items():
  print(k, v.shape)

input_ids torch.Size([4, 40])
token_type_ids torch.Size([4, 40])
attention_mask torch.Size([4, 40])
pixel_values torch.Size([4, 3, 384, 384])
pixel_mask torch.Size([4, 384, 384])
labels torch.Size([4, 6])


In [123]:
# Set the number of training epochs
num_epochs = 10

# Set the optimizer and learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Set the loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()

    # Initialize the total loss for this epoch
    total_loss = 0

    # Iterate over the batches in the train_dataloader
    for batch in train_dataloader:
        # Move the batch to the device
        batch = {key: value.to(device) for key, value in batch.items()}

        # Clear out the gradients from the previous batch
        optimizer.zero_grad()

        # Forward pass
        outputs = model(**batch)

        # Compute the loss
        loss = loss_fn(outputs.logits, batch['labels'])

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Accumulate the total loss for this epoch
        total_loss += loss.item()

    # Calculate the average loss for this epoch
    average_loss = total_loss / len(train_dataloader)

    # Print the average loss for this epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}")

# Training is complete
print("Training complete.")

Epoch 1/10, Average Loss: 1.5777
Epoch 2/10, Average Loss: 1.4051
Epoch 3/10, Average Loss: 1.2044
Epoch 4/10, Average Loss: 0.9747
Epoch 5/10, Average Loss: 0.7210
Epoch 6/10, Average Loss: 0.5099
Epoch 7/10, Average Loss: 0.4138
Epoch 8/10, Average Loss: 0.3027
Epoch 9/10, Average Loss: 0.2094
Epoch 10/10, Average Loss: 0.1594
Training complete.


#BCEloss

In [108]:
# Set the number of training epochs
num_epochs = 10


# Set the optimizer and learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Set the loss function
loss_fn = torch.nn.BCEWithLogitsLoss()
#loss_fn = nn.CrossEntropyLoss()
# Training loop
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()

    # Initialize the total loss for this epoch
    total_loss = 0

    # Iterate over the batches in the train_dataloader
    for batch in train_dataloader:
        # Move the batch to the device
        batch = {key: value.to(device) for key, value in batch.items()}

        # Clear out the gradients from the previous batch
        optimizer.zero_grad()

        # Forward pass
        outputs = model(**batch)
        m = torch.nn.Softmax()
        # print(outputs.logits.size(), m(outputs.logits).size())
        #print(torch.max(m(outputs.logits), 1))
        # print(torch.std_mean(m(outputs.logits)))
        # print(outputs.logits)

        # Compute the loss
        loss = loss_fn(outputs.logits, batch['labels'])

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Accumulate the total loss for this epoch
        total_loss += loss.item()

    # Calculate the average loss for this epoch
    average_loss = total_loss / len(train_dataloader)

    # Print the average loss for this epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}")

# Training is complete
print("Training complete.")

Epoch 1/10, Average Loss: 0.2250
Epoch 2/10, Average Loss: 0.2062
Epoch 3/10, Average Loss: 0.1910
Epoch 4/10, Average Loss: 0.1756
Epoch 5/10, Average Loss: 0.1600
Epoch 6/10, Average Loss: 0.1456
Epoch 7/10, Average Loss: 0.1281
Epoch 8/10, Average Loss: 0.1170
Epoch 9/10, Average Loss: 0.0955
Epoch 10/10, Average Loss: 0.0901
Training complete.


In [135]:
def calling_vilt_model(image, question, device):
    """
    Calling the ViLT model
    """
    test_encoding = processor(image, question, return_tensors="pt")
    test_encoding = {k: v.to(device) for k,v in test_encoding.items()}
    test_logits = model(**test_encoding).logits
    m = torch.nn.Sigmoid()
    # print(m(logits))
    # print(f"Standard deviation in confidence {torch.std_mean(m(logits)).cpu().numpy()}" )
    print(torch.max(m(test_logits)))
    print((m(test_logits)))
    answer = model.config.id2label[test_logits.argmax(-1).item()+1]
    print(test_logits.argmax(-1))
    print(torch.max(m(test_logits)))
    print((m(test_logits)))
    print("\n\033[1;31;34m>> Predicted answer =", answer)

    return answer

In [142]:
questions = "Which object is closest to the brown bowl, the blue block or the orange block?"
example_image = Image.open("/content/img5.jpg")
answer = calling_vilt_model(example_image, questions, device)

tensor(0.9950, device='cuda:0', grad_fn=<MaxBackward1>)
tensor([[0.5526, 0.9950, 0.6070, 0.0385, 0.0401, 0.3740]], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
tensor([1], device='cuda:0')
tensor(0.9950, device='cuda:0', grad_fn=<MaxBackward1>)
tensor([[0.5526, 0.9950, 0.6070, 0.0385, 0.0401, 0.3740]], device='cuda:0',
       grad_fn=<SigmoidBackward0>)

>> Predicted answer = Blue Block


# Train

In [ ]:
train_dataloader

In [ ]:
from tqdm.notebook import tqdm as T
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
model.train()

from torch.nn.utils.rnn import pad_sequence

# Assuming train_dataloader contains your training data

for epoch in range(3):
    print(f"Epoch: {epoch}")

    for batch in train_dataloader:
        # Transpose the batch to have a list of tensors for each input
        inputs = [torch.tensor(item).to(device) for item in zip(*batch)]

        # Pad the sequences to a uniform length
        inputs = pad_sequence(inputs, batch_first=True)

        # Get the input tensors from the padded batch
        input_ids, attention_masks, labels = inputs

        # Rest of your training code goes here...

        # get the inputs;
        batch = {k:v.to(device) for k,v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**batch)
        loss = outputs.loss
        print("Loss:", loss.item())
        loss.backward()
        optimizer.step()

Epoch: 0


ValueError: ignored

#Inference

In [ ]:
def calling_vilt_model(image, question, device):
    """
    Calling the ViLT model
    """
    encoding = processor(image, question, return_tensors="pt")
    encoding = {k: v.to(device) for k,v in encoding.items()}
    logits = model(**encoding).logits
    m = torch.nn.Sigmoid()
    print(m(logits))
    print(torch.std_mean(m(logits)))
    answer = model.config.id2label[logits.argmax(-1).item()]

    print("\n\033[1;31;34m>> Predicted answer =", answer)

    return answer

In [ ]:
questions = "Which object is closer to the middle, the yellow block or the gray block?"
example_image = Image.open("/content/img4.jpg")
answer = calling_vilt_model(example_image, questions, device)

tensor([[0.0018, 0.0026, 0.0026, 0.0021, 0.0020, 0.0028, 0.0024, 0.0023, 0.0019,
         0.0024, 0.0018, 0.0020, 0.0027, 0.0017, 0.0015, 0.0011, 0.0013, 0.0026,
         0.0023, 0.0028, 0.0020, 0.0022, 0.0019, 0.0020, 0.0033, 0.0010, 0.0022,
         0.0026, 0.0026, 0.0028, 0.0024, 0.0014, 0.0016, 0.0016, 0.0036, 0.0030,
         0.0024, 0.0017, 0.0024, 0.0021, 0.0020, 0.0019, 0.0022, 0.0013, 0.0031,
         0.0019, 0.0018, 0.0028, 0.0023, 0.0022, 0.0024, 0.0024, 0.0017, 0.0012,
         0.0018, 0.0015, 0.0032, 0.0018, 0.0025, 0.0016, 0.0028, 0.0020, 0.0022,
         0.0019, 0.0034, 0.0030, 0.0016, 0.0019, 0.0017, 0.0023, 0.0024, 0.0022,
         0.0025, 0.0018, 0.0025, 0.0023, 0.0019, 0.0028, 0.0017, 0.0023, 0.0023,
         0.0026, 0.0016, 0.0019, 0.0019, 0.0019, 0.0029, 0.0026, 0.0022, 0.0022,
         0.0020, 0.0018, 0.0015, 0.0025, 0.0021, 0.0014, 0.0018, 0.0016, 0.0018,
         0.0028, 0.0021, 0.0016, 0.0026, 0.0024, 0.0021, 0.0017, 0.0023, 0.0021,
         0.0018, 0.0022, 0.0

In [ ]:
example = examples
# add batch dimension + move to GPU
example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}

# forward pass
result = model(**example)

In [ ]:
result_logits = result.logits
print(torch.sigmoid(result_logits).size())
predicted_class = result_logits.argmax(-1).item()
m = torch.nn.Softmax()
print(m(result_logits))
print(torch.std_mean(m(result_logits)))
print(f"Predicted label: {predicted_class} Predicted answer: {model.config.id2label[predicted_class]}")

torch.Size([1, 153])
tensor([[0.0043, 0.0063, 0.0063, 0.0062, 0.0067, 0.0063, 0.0055, 0.0078, 0.0045,
         0.0062, 0.0071, 0.0063, 0.0081, 0.0067, 0.0063, 0.0074, 0.0092, 0.0048,
         0.0034, 0.0102, 0.0058, 0.0076, 0.0059, 0.0052, 0.0095, 0.0050, 0.0059,
         0.0063, 0.0052, 0.0103, 0.0066, 0.0094, 0.0077, 0.0043, 0.0065, 0.0043,
         0.0037, 0.0068, 0.0058, 0.0069, 0.0038, 0.0065, 0.0064, 0.0064, 0.0069,
         0.0067, 0.0036, 0.0049, 0.0080, 0.0078, 0.0040, 0.0065, 0.0061, 0.0056,
         0.0082, 0.0071, 0.0060, 0.0042, 0.0082, 0.0064, 0.0048, 0.0078, 0.0051,
         0.0062, 0.0057, 0.0075, 0.0058, 0.0047, 0.0035, 0.0072, 0.0066, 0.0042,
         0.0076, 0.0054, 0.0070, 0.0062, 0.0072, 0.0066, 0.0052, 0.0059, 0.0068,
         0.0055, 0.0070, 0.0076, 0.0103, 0.0060, 0.0062, 0.0070, 0.0059, 0.0074,
         0.0047, 0.0060, 0.0058, 0.0059, 0.0066, 0.0070, 0.0074, 0.0034, 0.0055,
         0.0078, 0.0062, 0.0056, 0.0085, 0.0082, 0.0079, 0.0055, 0.0068, 0.0059,
       

<ipython-input-97-46d14317492d>:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(m(result_logits))
<ipython-input-97-46d14317492d>:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.std_mean(m(result_logits)))


# GPT

In [ ]:
import openai
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
text_content = ""

image_descriptions = text_content
image_blocks = image_descriptions.split('\n\n')
output = ""

for image_block in image_blocks:
    image_name_start = image_block.index('Image Name: ')
    #print(image_name_start)
    image_name_end = image_block.index('.jpg')
    image_name = image_block[image_name_start:image_name_end]
    print(image_name)

    caption_start = image_block.index('Caption: ')
    caption = image_block[caption_start + len('Caption: '):].strip()
    print(f"Caption: {caption}")

    prompt = f"""ask five unique questions from this description: the questions should be strictly about two things. nothing else. These two things are:
    1. which object is where (for example: which object is closer to the middle? or top or which block farther from middle?),
    2. their relative distance (for example: which object is closest to the yellow bowl?).

    Don't ask questions that:
     1. may have mathematical values. For example, don't ask "how far is the yellow block from the red block?".
     2. which object is higher or lower. Assume all blocks and bowls are on a tabletop environment. They are all on a level surface.

    Ask the question for each captions: {caption}
    """
    response = get_completion(prompt)
    print(response)

In [ ]:
import torch
a = torch.randn(7)
print(a)
print(a.argmax(-1).item())

tensor([-1.1771,  0.4629, -0.5983, -0.4261, -0.9190, -1.6253, -0.4263])
1


In [41]:
import pandas as pd

def check_images_exist(csv_file):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file)

        # Create a list of expected image values from "img1" to "img107"
        expected_images = [f"img{i}" for i in range(1, 108)]

        # Get the unique values from the "image" column in the DataFrame
        unique_images = df['image'].unique()

        # Check if all expected image values exist in the "image" column
        missing_images = [image for image in expected_images if image not in unique_images]

        if not missing_images:
            print("All expected image values from 'img1' to 'img107' exist in the 'image' column.")
        else:
            print(f"Missing image values in the 'image' column: {', '.join(missing_images)}")
    except Exception as e:
        print(f"Error occurred: {e}")

# Usage example
if __name__ == "__main__":
    csv_file_path = "/content/sorted_output.csv"
    check_images_exist(csv_file_path)

Missing image values in the 'image' column: img67
